In [1]:
import polars as pl

splits = {'train': 'train.csv', 'validation': 'valid.csv'}
df = pl.read_csv('hf://datasets/COGNANO/VHHCorpus-2M/' + splits['train'])

df2 = df.with_columns(n_cys=pl.col('VHH_sequence').str.count_matches('C'), 
                      n_glycosilations=pl.col('VHH_sequence').str.count_matches('N[^P][TS]')).select(
    sequence=pl.col('VHH_sequence'), suitable_for_expression=pl.all_horizontal(pl.col('n_glycosilations')==0, 
                                                                               pl.col('n_cys') == 2))

df2 = df2.lazy().with_columns(pl.col("suitable_for_expression").cast(int).alias("target")).collect()
df2 = df2[['sequence', 'target']]
df2.write_csv('llama_2M.tsv', separator='\t')


In [2]:
df2.get_column('target').value_counts()

target,count
i64,u32
0,834119
1,1165881


In [3]:
df2.sample(n=2000, seed=0).write_csv('llama_2K.tsv', separator='\t')
df2.sample(n=200000, seed=0).write_csv('llama_200K.tsv', separator='\t')

In [4]:
df2.sample(n=200, seed=0).get_column('target').value_counts()

target,count
i64,u32
1,121
0,79


In [5]:
df2.sample(n=2000, seed=0).head()

sequence,target
str,i64
"""MKYLLPTAAAGLLLLAAQPAMAQVQLQESG…",1
"""MKYLLPTAAAGLLLLAAQPAMAQVQLQESG…",1
"""MKYLLPTAAAGLLLLAAQPAMAQVQLQESG…",1
"""MKYLLPTAAAGLLLLAAQPAMAQAQLQESG…",1
"""MKYLLPTAAAGLLLLAAQPAMAQVQLQESG…",0
